In [ ]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

In [ ]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.



In [ ]:
def create_singlet():
    """
    Create a 2-qubit circuit that prepares the singlet state
    """
    qc = QuantumCircuit(2)
    qc.x(1)          
    qc.h(0)
    qc.cx(0, 1)
    qc.z(1)         
    return qc

def measure_in_basis(qc, qubit, basis):
    """
    Rotate the qubit so that a subsequent Z measurement is equivalent 
    to measuring in the desired basis.
     """
    if basis == 'Z':
        pass
    elif basis == 'X':
        qc.h(qubit)
    elif basis == 'V':
        qc.ry(-math.pi/2, qubit)
    else:
        raise ValueError("Unknown basis provided.")

def basis_selector():
    """
    Use a quantum coin toss to choose a basis with bias
    so that outcome 0 occurs with probability 1/3 and outcome 1 with 2/3.
    """
    theta = 2 * math.acos(1/math.sqrt(3))  # Ensures cos²(theta/2)=1/3.
    qc = QuantumCircuit(1, 1)
    qc.ry(theta, 0)
    qc.measure(0, 0)
    
    sim = BasicSimulator()
    job = sim.run(qc, shots=1)
    result = job.result()
    counts = result.get_counts(qc)
    outcome = list(counts.keys())[0]
    return int(outcome)

def simulate_round():
    """
    Simulate one round of the plain Ekert91 protocol.
    """
    qc = create_singlet()
    
    # Basis selection for Alice and Bob.
    alice_coin = basis_selector() 
    bob_coin   = basis_selector()   
    alice_basis = 'Z' if alice_coin == 0 else 'X'
    bob_basis   = 'Z' if bob_coin == 0 else 'V'
    
    # Apply rotations so that a Z measurement effectively measures in the chosen basis.
    measure_in_basis(qc, 0, alice_basis)
    measure_in_basis(qc, 1, bob_basis)
    
    # Add measurement operations.
    qc.measure_all()
    
    sim = BasicSimulator()
    job = sim.run(qc, shots=1)
    result = job.result()
    counts = result.get_counts(qc)
    outcome = list(counts.keys())[0]
    
    # bit ordering: rightmost bit is qubit 0 (Alice), next is qubit 1 (Bob).
    alice_result = int(outcome[-1])
    bob_result   = int(outcome[-2])
    
    return alice_basis, bob_basis, alice_result, bob_result

def simulate_protocol(num_rounds=100):
 
    results = []
    for _ in range(num_rounds):
        results.append(simulate_round())
    return results

if __name__ == "__main__":
    rounds = simulate_protocol(num_rounds=10)
    print("Example round (plain protocol):", rounds[0])